In [ ]:
import pandas as pd
import numpy as np
import math
import sys

df = pd.read_csv('train.csv')
topk = 10

record_count = df.shape[0]

users = pd.unique(df["user"])
num_users = len(users)

new_records = num_users * topk

df2 = df[["continent"]]
df2 = df2.groupby(by=["continent"]).size()
df2 = df2.reset_index()
df2.columns = ["continent", "count_continent"]

df2["percent_continent"] = df2["count_continent"] / record_count

df2["new_continents"] = df2["percent_continent"] * new_records
df2['new_continents'] = df2['new_continents'].apply(round)

df2['percent_continent'] = df2['percent_continent'].apply(lambda x: '%.16f' % x)

df2 = df2.sort_values(
    by=['percent_continent'],
    ascending=[True])

sum_records = df2["new_continents"].sum()

difference = new_records - sum_records

row_count = df2.shape[0]

df2.iat[row_count-1, 3] = df2.iat[row_count-1, 3] + difference

df2 = df2[["continent", "percent_continent","new_continents"]]

df2.to_csv("porc-cont.csv", index=False,  header=True)

print("\nSaving... process finished")

In [ ]:
import pandas as pd
import numpy as np
import sys

df = pd.read_csv('train.csv')
topk = 10

record_count = df.shape[0]

users = pd.unique(df["user"])
num_users = len(users)

new_records = num_users * topk

df2 = df[["country"]]
df2 = df2.groupby(by=["country"]).size()
df2 = df2.reset_index()
df2.columns = ["country", "count_country"]

df2["percent_country"] = df2["count_country"] / record_count

df2["new_countries"] = df2["percent_country"] * new_records
df2['new_countries'] = df2['new_countries'].apply(round)

df2['percent_country'] = df2['percent_country'].apply(lambda x: '%.16f' % x)

df2 = df2.sort_values(
    by=['percent_country'],
    ascending=[True])

sum_records = df2["new_countries"].sum()

difference = new_records - sum_records

row_count = df2.shape[0]

df2.iat[row_count-1, 3] = df2.iat[row_count-1, 3] + difference

df2 = df2[["country", "percent_country","new_countries"]]

df2.to_csv("porc-country.csv", index=False,  header=True)

print("\nSaving... process finished")

In [ ]:
import pandas as pd
import logging
import math
from datetime import datetime
import numpy as np
from sklearn.preprocessing import minmax_scale

np.set_printoptions(threshold=3000)
np.set_printoptions(linewidth=100)
np.set_printoptions(edgeitems=10)

now = datetime.now()
print(now)

pd.options.mode.chained_assignment = None  # Default='warn'


algorithm = 'ItemKNN'
df = pd.read_csv(algorithm +'.csv')
topk = 10
topn = 1000 

# Normalize rating (0.0-1.0 range)
df['rating'] = minmax_scale(df['rating'])
df = df.to_numpy()

record_count = df.shape[0]
df = np.column_stack((df, np.zeros(record_count)))
df = np.column_stack((df, np.zeros(record_count)))

index = [0] * record_count

for i in range(record_count):
    base = math.floor(i / topn)
    inicio = base * topn
    index[i] = inicio + topk - 1

col_rating=3
col_pos = 4
base_loss = 7
loss = 8

df[:, base_loss] = df[index, col_rating]
df[:, loss] = (df[:, col_rating] - df[:, base_loss]) / (df[:, base_loss] + 0.0000000000000001)

# Define by zero (0.0) the lost where the position is lesser than the topk value
indexes = np.where(df[:, col_pos] < topk)
df[indexes, loss] = 0

# Drop the base-loss column
df = np.delete(df,base_loss,1)

df2 = pd.DataFrame(data=df,
                   columns=["id",
                        "user",
                        "item",
                        "rating",
                        "position",
                        "continent",
                        "country",  
                      # "base_loss",
                        "loss"
                        ])

loss_max = min(df2["loss"])
print('loss_max:', loss_max)

df2.to_csv(algorithm +"-loss.csv", index=False,  header=True)

now = datetime.now()
print(now)

print("\nSaving... process finished")

In [ ]:
import pandas as pd
import numpy as np
import sys

algorithm = 'ItemKNN'
df = pd.read_csv(algorithm+'-loss.csv')

df_cont = pd.read_csv('porc-cont.csv')
df_country = pd.read_csv('porc-country.csv')

df_cont = df_cont[["continent","percent_continent","new_continents"]]
df_country = df_country[["country", "percent_country", "new_countries"]] 

df2 = pd.merge(df, df_cont, how="left", on=["continent"])
df2 = pd.merge(df2, df_country, how="left", on=["country"])

df2 = df2.sort_values(
    by=['percent_continent',"continent", "percent_country","country","user", "rating"],
    ascending=[True, True, True, True, True, False])

df2.to_csv(algorithm+"-join.csv", index=False, header=True)

print("\nSaving... process finished")

In [ ]:
from re import S
import pandas as pd
import numpy as np
import sys
import logging

# Logging.basicconfig(
    # Format='%(asctime)s -> %(message)s',
    # Level=logging.info,
    # Handlers=[logging.filehandler("calibracion.log"),
              # Logging.streamhandler()]
# )

# Np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(threshold=3000)
np.set_printoptions(linewidth=100)
np.set_printoptions(edgeitems=10)

col_id = 0
col_user = 1
col_item = 2
col_rating = 3
col_pos = 4
col_cont = 5
col_country = 6
col_loss = 7
col_percent_continent = 8
new_continents = 9
col_percent_country = 10
new_countries = 11
col_priority = 12
col_cu = 13
col_cc = 14
col_ccn = 15
col_fase = 16
col_destino = 17

columns = ['id',
           'user',
           'item',
           'rating',
           'position',
           'continent',
           'country',
           'loss',
           'percent_continent',
           'new_continents',
           'percent_country',
           'new_countries',
           'priority',
           'cu',
           'cc',
           'ccn',
           'Fase',
           'Destino']

tolerance = 1
algorithm = 'ItemKNN'
df = pd.read_csv(algorithm+'-join.csv')
topk = 10

df['priority'] = df['percent_country'].apply(lambda x: 3 if x <=0.1 else 2)
df['is_max'] = df['rating'].eq((df.loc[df['priority'] == 3]).groupby('user')['rating'].transform('max'))
df.loc[df['is_max'] == True,'priority'] = 1

df['rating_2'] = df['rating']
df.loc[df['priority'] == 1,'rating_2' ] = 1  
df.loc[df['priority'] == 2,'rating_2' ] = 1  

df = df.sort_values(
    by=['priority','rating_2','percent_continent', "continent", "percent_country", "country", "user", "rating"],
    ascending=[True, False, True, True, True, True, True, False])

del df["is_max"]
del df["rating_2"]

df = df.to_numpy()

record_count = df.shape[0]

logging.info("Realizando configuración básica...")
df = np.column_stack((df, np.zeros(record_count)))
df = np.column_stack((df, np.zeros(record_count)))
df = np.column_stack((df, np.zeros(record_count)))
df = np.column_stack((df, np.zeros(record_count)))
df = np.column_stack((df,  [' '] * record_count))

df_cont = pd.read_csv('porc-cont.csv')
df_cont = df_cont.to_numpy()
record_count_cont = df_cont.shape[0]

df_country = pd.read_csv('porc-country.csv')
df_country = df_country.to_numpy()
record_count_country = df_country.shape[0]

df[:, col_loss] = abs(df[:, col_loss])  

users = np.unique(df[:, col_user])
users_count = dict(zip(users, np.zeros(len(users))))

countries = np.unique(df[:, col_country])
countries_count = dict(zip(countries, np.zeros(len(countries))))

continents = np.unique(df[:, col_cont])
continents_count = dict(zip(continents, np.zeros(len(continents))))

new_rows_country = dict(zip(df_country[:, 0].tolist(), df_country[:, 2].tolist()))

new_rows_count = dict(zip(df_cont[:, 0].tolist(), df_cont[:, 2].tolist()))

x=1
num_users = len(users)

logging.info("Phase 1:")
for i in range(record_count):
    row = df[i]
    
    if(row[col_loss] <= tolerance):
        user = row[col_user]
        country = row[col_country]
        continent = row[col_cont]

        if (country in new_rows_country) and (continent in new_rows_count):
            cu = users_count[user] + 1
            cc = countries_count[country] + 1
            ccn = continents_count[continent] + 1

            if(new_rows_country[country] >= cc and
                    new_rows_count[continent] >= ccn and
                    topk >= cu):

                row[col_cu] = cu
                row[col_cc] = cc
                row[col_ccn] = ccn

                row[col_fase] = 1  # Phase 1
                row[col_destino] = str(user) + "-" + country + "-" + continent

                users_count[user] = cu
                countries_count[country] = cc
                continents_count[continent] = ccn
                
                df[i] = row
    
    if(i % 10000 == 0):
        logging.info(str(i) + " / " + str(record_count) + ": " +
                     str(round(i/record_count*100, 2)) + "%")

logging.info("Phase 2:")
for i in range(record_count):
    row = df[i]
    
    if(row[col_loss] <= tolerance and row[col_fase] == 0):
        user = row[col_user]
        country = row[col_country]
        continent = row[col_cont]

        if (continent in new_rows_count):
            cu = users_count[user] + 1
            ccn = continents_count[continent] + 1

            if(new_rows_count[continent] >= ccn and topk >= cu):
                row[col_cu] = cu
                row[col_ccn] = ccn

                row[col_fase] = 2  # Phase 2
                row[col_destino] = str(user) + "-" + country + "-" + continent

                users_count[user] = cu
                continents_count[continent] = ccn

                df[i] = row

    if(i % 10000 == 0):
        logging.info(str(i) + " / " + str(record_count) + ": " +
                     str(round(i/record_count*100, 2)) + "%")

logging.info("Phase 3:")
for i in range(record_count):
    row = df[i]

    if(row[col_loss] <= tolerance and row[col_fase] == 0):
        user = row[col_user]
        country = row[col_country]
        continent = row[col_cont]

        if (True):
            cu = users_count[user] + 1
            
            if(topk >= cu):
                row[col_cu] = cu
                
                row[col_fase] = 3  # Phase 3
                row[col_destino] = str(user) + "-" + country + "-" + continent

                users_count[user] = cu
                
                df[i] = row

    if(i % 10000 == 0):
        logging.info(str(i) + " / " + str(record_count) + ": " +
                     str(round(i/record_count*100, 2)) + "%")

indexes = np.where(df[:, col_fase]  == 0)
df = np.delete(df, indexes, 0)
  
    
logging.info(users_count)

logging.info("Sorting out...")

df2 = pd.DataFrame(data=df, columns=columns)

df2 = df2.sort_values(
   by=['user', 'rating'],
   ascending=[True, False])

df = df2.to_numpy()

logging.info("Saving results...")
df2 = pd.DataFrame(data=df, columns=columns)

df2['percent_continent'] = df2['percent_continent'].apply(
    lambda x: '%.16f' % x)

df2['percent_country'] = df2['percent_country'].apply(
    lambda x: '%.16f' % x)

df2['loss'] = df2['loss'].apply(
    lambda x: '%.16f' % x)

df2.to_csv(algorithm +'-'+str(round(tolerance*100))+'.csv', index=False, header=True)

print("\nSaving... process finished")